<a href="https://colab.research.google.com/github/DavidTatis/HyperParameter-Tunning/blob/main/GridSearch_NN_IRNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libreries

In [ ]:
!pip install scikeras[tensorflow]
!pip install scikeras[tensorflow-cpu]

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # plotting library
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation, Dropout,BatchNormalization,Input
from tensorflow.keras.optimizers import Adam ,RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras import  backend as K
from sklearn.model_selection import GridSearchCV
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
import numpy as np
from numpy import genfromtxt
from google.colab import drive
import glob
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, Input,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import KFold





Dataset

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
train_X = genfromtxt("/content/drive/My Drive/AI datasets/elements/train_X.csv", delimiter=',')
train_y = genfromtxt("/content/drive/My Drive/AI datasets/elements/train_Y.csv", delimiter=',')
valid_X = genfromtxt("/content/drive/My Drive/AI datasets/elements/valid_X.csv", delimiter=',')
valid_y = genfromtxt("/content/drive/My Drive/AI datasets/elements/valid_y.csv", delimiter=',')
test_X = genfromtxt("/content/drive/My Drive/AI datasets/elements/test_X.csv", delimiter=',')
test_y = genfromtxt("/content/drive/My Drive/AI datasets/elements/test_y.csv", delimiter=',')


Mounted at /content/drive


In [ ]:
#num_layers refers to the layers in each block of the network. See models summary to clarify
def irnet_model(num_layers,units,units_decrease,activation_function,optimizer):

    input_vec = Input(shape=(86,))
    x1=Activation(activation_function)(BatchNormalization()(Dense(units)(input_vec)))
    m1=concatenate([input_vec, x1], axis=-1)
    for layer in range(num_layers-1):
      x1=Activation('relu')(BatchNormalization()(Dense(units)(m1)))
      m1=concatenate([m1, x1], axis=-1)
    
    
    units=int(units/units_decrease)
    
    x2=Activation(activation_function)(BatchNormalization()(Dense(units)(m1)))
    m2=concatenate([m1, x2], axis=-1)
    for layer in range(num_layers-1):
      x2=Activation(activation_function)(BatchNormalization()(Dense(units)(m2)))
      m2=concatenate([m2, x2], axis=-1)
    
    units=int(units/units_decrease)
      
    x3=Activation(activation_function)(BatchNormalization()(Dense(units)(m2)))
    m3=concatenate([m2, x3], axis=-1)
    for layer in range(num_layers-1):
      x3=Activation(activation_function)(BatchNormalization()(Dense(units)(m3)))
      m3=concatenate([m3, x3], axis=-1)
    

    x4=Activation(activation_function)(BatchNormalization()(Dense(64)(m3)))
    m4=concatenate([m3, x4], axis=-1)
    x5=Activation(activation_function)(BatchNormalization()(Dense(32)(m4)))
    
    
    m5=concatenate([m4, x5], axis=-1)
    x6=Dense(1,activation='linear')(m5)
    
    model=Model(input_vec,x6)
    model.summary()
    if(optimizer.lower()=='adam'):
      model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=tf.keras.losses.mean_absolute_error,
        metrics=[tf.keras.metrics.MeanAbsoluteError()])
    elif(optimizer.lower()=='sgd'):
       model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001,momentum=0.9),
        loss=tf.keras.losses.mean_absolute_error,
        metrics=[tf.keras.metrics.MeanAbsoluteError()])
    plot_model(model,to_file='model.png')
    return model

K-Cross Validation

In [ ]:
k_folds=10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=1234)
train_Xi=[[] for each in range(k_folds)]
train_Yi=[[] for each in range(k_folds)]
valid_Xi=[[] for each in range(k_folds)]
valid_Yi=[[] for each in range(k_folds)]
i=0

for train_index, val_index in kf.split(train_X):
    train_Xi[i] = pd.DataFrame(train_X).iloc[train_index,]
    train_Yi[i] = pd.DataFrame(train_y).iloc[train_index,]
    
    valid_Xi[i]= pd.DataFrame(train_X).iloc[val_index,]
    valid_Yi[i]= pd.DataFrame(train_y).iloc[val_index,]

    i=i+1

In [ ]:
i=0 #for tests and development porpose im using only the fold 0.
irnet_model=irnet_model(num_layers=1,units=2048,units_decrease=2,activation_function='relu',optimizer='adam')
batch_size=16

In [ ]:
# create model
from scikeras.wrappers import KerasClassifier
model = KerasClassifier(build_fn=irnet_model,epochs=10, batch_size=batch_size, verbose=0,
                        num_layers=1,units=2048,units_decrease=2,activation_function='relu',optimizer='adam')
# define the grid search parameters
num_layers = [2,3,4]
units_decrease = [2,4]
activation_function=['relu','tanh']
optimizer=['adam']
units=[2048,1024]
param_grid = dict(num_layers=num_layers,units_decrease=units_decrease,activation_function=activation_function,optimizer=optimizer,units=units)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(train_Xi[i], train_Yi[i]) #TODO currently a memory when fitting (searching in google, its related to a version of Tensorflow)
